In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from pandas.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.optimize import minimize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
## !pip install 'polars[numpy,pandas,pyarrow]' --target=/kaggle/working/mysitepackages

!pip download polars[numpy,pandas,pyarrow] -d /kaggle/working/mysitepackages/polars_pkg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 32.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 22.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 53.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 18.4 MB/s eta 0:00:00
Saved ./mysitepackages/polars_pkg/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Saved ./mysitepackages/polars_pkg/pyarrow-12.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/polars_pkg/typing_extensions-4.5.0-py3-none-any.whl
Saved ./mysitepackages/polars_pkg/pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./mysitepackages/polars_pkg/polars-0.17.12-cp37-abi3-manylinux_2_17_x86_64.m

In [3]:
!pip download duckdb==0.7.1 -d /kaggle/working/mysitepackages/duck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 30.2 MB/s eta 0:00:0000:0100:01
Saved ./mysitepackages/duck/duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Successfully downloaded duckdb


#### create the zip file with the code below and then download the zip file into local machine

In [4]:
from zipfile import ZipFile

dirName = "/kaggle/working/mysitepackages"
zipName = "packages.zip"

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)

#### Download the zipfile into local machine
#### Upload the downloaded zipfile as data from local machine and then fire the commands below

In [6]:
! ls /kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages

duck  polars_pkg


In [7]:
!pip install polars[numpy,pandas,pyarrow] --no-index --find-links=file:///kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages/polars_pkg

Looking in links: file:///kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages/polars_pkg


In [2]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages/duck

Looking in links: file:///kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages/duck
Processing /kaggle/input/duckdb-n-polars-offline/kaggle/working/mysitepackages/duck/duckdb-0.7.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [3]:
import polars as pl
import duckdb as dd

In [4]:
defog_metadata = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv", try_parse_dates=True)\
.collect()
print(defog_metadata.shape)
print(defog_metadata.select('Subject').n_unique())
print(defog_metadata.select('Id').n_unique())

subjects = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv", try_parse_dates=True)\
.collect()
print(subjects.shape)
print(subjects.select('Subject').n_unique())

defog_subjects_metadata = defog_metadata.join(subjects, on=["Subject","Visit"], how="inner")

defog_subjects_metadata = defog_subjects_metadata.with_columns(pl.when(pl.col('Medication') == 'on').then(pl.col('UPDRSIII_On')).otherwise(pl.col('UPDRSIII_Off')).alias("UPDRSIII"))
defog_subjects_metadata = defog_subjects_metadata.drop(['UPDRSIII_On','UPDRSIII_Off'])

del defog_metadata, subjects

print(defog_subjects_metadata.shape)
print(defog_subjects_metadata.select('Subject').n_unique())
print(defog_subjects_metadata.select('Id').n_unique())

(137, 4)
45
137
(173, 8)
136
(137, 9)
45
137


In [5]:
events = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv", try_parse_dates=True)
events = events.with_columns(
    [
        ((pl.col('Init')*100).round(0).cast(pl.Int64)).alias('Init_m'), 
        ((pl.col('Completion')*100).round(0).cast(pl.Int64)).alias('Completion_m')
    ]
)
events = events.with_columns(
    [
        (pl.when(pl.col('Type')=='Turn').then(pl.lit(1)).otherwise(pl.lit(0)).alias('Turn')),
        (pl.when(pl.col('Type')=='Walking').then(pl.lit(1)).otherwise(pl.lit(0)).alias('Walking')),
        (pl.when(pl.col('Type')=='StartHesitation').then(pl.lit(1)).otherwise(pl.lit(0)).alias('StartHesitation'))
    ]
)

events = events.drop(['Init','Completion','Type'])
events = events.rename({'Init_m': 'Init', 'Completion_m':'Completion'})

events = events.sort(['Id','Init','Completion'], nulls_last=True)

In [8]:
"""## polars SQL can only be used on one dataframe alone, joins are not yet supported
plsql = pl.SQLContext()
#temp_defog = pl.scan_csv(os.path.join(dirname, filename), try_parse_dates=True).filter((pl.col("Valid")) & (pl.col("Task"))).with_columns(pl.lit(filename[0:-4]).alias('Id'))
plsql.register(name='temp_defog', lf=temp_defog)
plsql.register(name='events', lf=events)
temp = plsql.execute(query='select * from temp_defog, events where temp_defog.Id=events.Id limit 10')
temp_df = temp.collect()
temp_df.filter((pl.col('Time')>=np.round(377.635 * 100)) & (pl.col('Time')<=np.round(387.785 * 100)))"""

In [6]:
tasks = pl.scan_csv("/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv", try_parse_dates=True)

tasks = tasks.with_columns(
    [
        ((pl.col('Begin')*100).round(0).cast(pl.Int64)).alias('Begin_m'), 
        ((pl.col('End')*100).round(0).cast(pl.Int64)).alias('End_m')
    ]
)

tasks = tasks.drop(['Begin','End'])
tasks = tasks.rename({'Begin_m': 'Begin', 'End_m':'End'})

"""print(tasks.shape)
print(tasks.select('Id').n_unique())
tasks.filter(pl.col('Id').is_in(['a8c5a489bd']))"""

"print(tasks.shape)\nprint(tasks.select('Id').n_unique())\ntasks.filter(pl.col('Id').is_in(['a8c5a489bd']))"

In [7]:
defog_train_data = pl.DataFrame()

for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/defog'):
    for filename in filenames:
        
        temp_defog = pl.scan_csv(os.path.join(dirname, filename), try_parse_dates=True).filter((pl.col("Valid")) & (pl.col("Task"))).with_columns(pl.lit(filename[0:-4]).alias('Id'))
        temp_defog = temp_defog.with_columns(
            [
                ((pl.col('AccV')*9.80665).round(5).cast(pl.Float64)).alias('AccV_ms2'), 
                ((pl.col('AccML')*9.80665).round(5).cast(pl.Float64)).alias('AccML_ms2'),
                ((pl.col('AccAP')*9.80665).round(5).cast(pl.Float64)).alias('AccAP_ms2')
            ]
        )
        temp_defog = temp_defog.drop(['AccV','AccML','AccAP'])
        temp_defog = temp_defog.rename({'AccV_ms2': 'AccV', 'AccML_ms2':'AccML', 'AccAP_ms2':'AccAP'})
        
        temp_defog = temp_defog.sort(['Id','Time'], nulls_last=True)
        
        temp_final = dd.sql('SELECT td.Id, td.Time, td.AccV, td.AccML, td.AccAP, td.StartHesitation, td.Turn, td.Walking, coalesce(e.Kinetic,-1) as Kinetic, t.Task \
        FROM temp_defog td \
        left join events e on td.Id = e.Id and td.Time >= e.Init and td.Time <= e.Completion \
        left join tasks t on td.Id = t.Id and td.Time >= t.Begin and td.Time <= t.End').pl()
        
        defog_train_data = pl.concat([defog_train_data, temp_final], how="vertical")
        del temp_final
        
defog_train_data_with_metadata = dd.sql('select t1.Id, t2.Subject, t2.Visit, t1.Time, t2.Medication, t2.Age, t2.Sex, t2.YearsSinceDx \
,t2.NFOGQ, t2.UPDRSIII, t1.Task, t1.AccV, t1.AccML, t1.AccAP, t1.Kinetic \
,t1.StartHesitation, t1.Turn, t1.Walking \
from defog_train_data t1 join defog_subjects_metadata t2 on t1.Id = t2.Id').pl()

del defog_train_data, defog_subjects_metadata
gc.collect()

60

In [8]:
defog_train_data_with_metadata.write_parquet('/kaggle/working/defog_train_data_with_metadata.parquet')

In [29]:
defog_train_data_with_metadata.head(10)

Id,Subject,Visit,Time,Medication,Age,Sex,YearsSinceDx,NFOGQ,UPDRSIII,Task,AccV,AccML,AccAP,Kinetic,StartHesitation,Turn,Walking
str,str,i64,i64,str,i64,str,f64,i64,i64,str,f64,f64,f64,i64,i64,i64,i64
"""77d7d95074""","""413532""",1,12721,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.16006,-2.65337,5.18014,-1,0,0,0
"""77d7d95074""","""413532""",1,12722,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.17523,-2.65716,5.16594,-1,0,0,0
"""77d7d95074""","""413532""",1,12723,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.18099,-2.64263,5.19295,-1,0,0,0
"""77d7d95074""","""413532""",1,12724,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.18064,-2.62481,5.18995,-1,0,0,0
"""77d7d95074""","""413532""",1,12725,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.16846,-2.63602,5.18171,-1,0,0,0
"""77d7d95074""","""413532""",1,12726,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.15146,-2.63611,5.17084,-1,0,0,0
"""77d7d95074""","""413532""",1,12727,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.18233,-2.63937,5.16944,-1,0,0,0
"""77d7d95074""","""413532""",1,12728,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.19326,-2.64533,5.17349,-1,0,0,0
"""77d7d95074""","""413532""",1,12729,"""on""",77,"""M""",21.0,24,37,"""Rest1""",-8.1694,-2.63998,5.1692,-1,0,0,0


In [30]:
defog_train_data_with_metadata.describe()

describe,Id,Subject,Visit,Time,Medication,Age,Sex,YearsSinceDx,NFOGQ,UPDRSIII,Task,AccV,AccML,AccAP,Kinetic,StartHesitation,Turn,Walking
str,str,str,f64,f64,str,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""4090539""","""4090539""",4.090539e6,4.090539e6,"""4090539""",4.090539e6,"""4090539""",4.090539e6,4.090539e6,4.090539e6,"""4090539""",4.090539e6,4.090539e6,4.090539e6,4.090539e6,4.090539e6,4.090539e6,4.090539e6
"""null_count""","""0""","""0""",0.0,0.0,"""0""",0.0,"""0""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,1.350975,91125.222825,null,67.326863,null,11.79858,20.868,41.21466,null,-9.16869,-0.012051,-2.214143,-0.728031,0.000122,0.14346,0.024084
"""std""",null,null,0.477275,70259.156001,null,8.229973,null,6.737574,3.732369,12.018071,null,1.051508,1.374085,2.397416,0.637287,0.011055,0.350541,0.153311
"""min""","""02ea782681""","""00f674""",1.0,1000.0,"""off""",28.0,"""F""",1.5,12.0,13.0,"""4MW""",-59.08213,-20.74114,-50.19681,-1.0,0.0,0.0,0.0
"""max""","""f9fc61ce85""","""f28337""",2.0,414387.0,"""on""",82.0,"""M""",30.0,27.0,76.0,"""Turning-ST""",43.72162,44.36565,43.03288,1.0,1.0,1.0,1.0
"""median""",null,null,1.0,76822.0,null,69.0,null,10.0,22.0,43.0,null,-9.21079,0.0,-2.45166,-1.0,0.0,0.0,0.0


In [31]:
defog_train_data_with_metadata.select('AccV','AccML','AccAP','Kinetic','Age','NFOGQ','UPDRSIII','StartHesitation','Turn','Walking').corr()

AccV,AccML,AccAP,Kinetic,Age,NFOGQ,UPDRSIII,StartHesitation,Turn,Walking
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,-0.05039,-0.176834,0.064818,0.000259,0.062652,0.012789,-0.000539,0.079231,-0.027536
-0.05039,1.0,-0.12909,0.020651,-0.147448,-0.063807,0.043725,-0.002988,0.032275,-0.00538
-0.176834,-0.12909,1.0,-0.184135,0.056648,-0.105752,-0.096863,-0.000724,-0.203901,-0.002958
0.064818,0.020651,-0.184135,1.0,0.024269,0.141274,0.155681,0.027864,0.845913,0.376652
0.000259,-0.147448,0.056648,0.024269,1.0,-0.088312,-0.115594,-0.002521,0.062243,-0.071971
0.062652,-0.063807,-0.105752,0.141274,-0.088312,1.0,0.453715,-0.001386,0.169732,0.038972
0.012789,0.043725,-0.096863,0.155681,-0.115594,0.453715,1.0,0.00415,0.171918,0.075783
-0.000539,-0.002988,-0.000724,0.027864,-0.002521,-0.001386,0.00415,1.0,-0.004525,-0.001737
0.079231,0.032275,-0.203901,0.845913,0.062243,0.169732,0.171918,-0.004525,1.0,-0.064291


In [11]:
dd.sql('select Task, Visit, count(1) from defog_train_data_with_metadata group by Task, Visit').pl().head(65).transpose()

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49,column_50,column_51,column_52,column_53,column_54
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Hotspot1-C""","""Hotspot1""","""Hotspot2""","""Hotspot2-C""","""MB3-R""","""Turning-C""","""Turning-ST""","""Rest1""","""Rest2""","""4MW""","""TUG-ST""","""TUG-DT""","""Turning-ST""","""Turning-DT""","""Hotspot1""","""Hotspot2""","""4MW-C""","""4MW""","""TUG-ST""","""TUG-C""","""TUG-DT""","""Turning-DT""","""MB3-L""","""MB2b""","""MB1""","""MB4""","""MB6-R""","""MB5""","""MB7""","""MB6-L""","""MB8""","""MB9""","""MB13""","""MB12""","""MB11""","""MB10""","""MB2b""","""MB1""","""MB4""","""MB3-R""","""MB3-L""","""MB6-L""","""MB6-R""","""MB5""","""MB7""","""MB9""","""MB8""","""MB11""","""MB10""","""MB13""","""MB12""","""MB2a""","""MB2a""","""Hotspot1-C""","""MB6"""
"""2""","""2""","""2""","""2""","""2""","""2""","""2""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""2""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""2""","""1""","""1"""
"""44606""","""51284""","""124647""","""97484""","""27702""","""109770""","""184865""","""271045""","""90783""","""106596""","""201758""","""243007""","""341096""","""418336""","""111284""","""360231""","""42413""","""40821""","""80827""","""68905""","""114040""","""238492""","""30487""","""8368""","""5897""","""4928""","""5682""","""6924""","""37261""","""6243""","""18861""","""37782""","""10283""","""10313""","""10153""","""13436""","""16961""","""14920""","""14637""","""45183""","""55057""","""12240""","""12733""","""14033""","""80411""","""79602""","""51137""","""29810""","""31405""","""22147""","""19458""","""8865""","""3202""","""1700""","""428"""


In [25]:
defog_task_list = defog_train_data_with_metadata.get_column('Task').unique().to_list()

In [34]:

temp.head(10)

Time,AccV,Sex,StartHesitation
i64,f64,str,str
97653,-6.55327,"""M""","""0"""
97654,-7.95385,"""M""","""0"""
97655,-6.82419,"""M""","""0"""
97656,-8.943,"""M""","""0"""
97657,-11.16464,"""M""","""0"""
97658,-12.05456,"""M""","""0"""
97659,-13.79052,"""M""","""0"""
97660,-14.21803,"""M""","""0"""
97661,-12.66816,"""M""","""0"""


In [35]:
temp_pdf = temp.to_pandas()
fig = px.scatter(temp_pdf, x='Time', y='AccV', color='StartHesitation', facet_col="Sex")

In [36]:
fig.show()

In [39]:
fig_dict = {}
for task in defog_task_list:
    temp = defog_train_data_with_metadata.filter((pl.col('Visit')==1) & (pl.col('Task')==task)).select('Time','AccV','StartHesitation','Sex')
    temp = temp.with_columns(
        [
            (pl.col('StartHesitation').cast(pl.Utf8)).alias('StartHesitation_str')
        ]
    )

    temp = temp.drop(['StartHesitation'])
    temp = temp.rename({'StartHesitation_str': 'StartHesitation'})
    #print(temp.shape)
    #print(temp.columns)
    temp_pdf = temp.to_pandas()
    if temp.shape[0] != 0:
        print('For Task :', task)
        fig = px.scatter(temp_pdf, x="Time", y="AccV", color="StartHesitation", facet_col="Sex")
        fig_dict[task] = fig
    #del temp, temp_pdf
    #gc.collect()

For Task : MB5
For Task : TUG-DT
For Task : MB7
For Task : Turning-DT
For Task : MB2b
For Task : MB13
For Task : MB1
For Task : Rest1
For Task : Rest2
For Task : Hotspot1-C
For Task : MB12
For Task : Hotspot2
For Task : MB2a
For Task : TUG-ST
For Task : MB9
For Task : MB8
For Task : MB6-R
For Task : Turning-ST
For Task : MB3-L
For Task : MB10
For Task : MB6-L
For Task : 4MW
For Task : MB4
For Task : MB11
For Task : MB6
For Task : MB3-R
For Task : Hotspot1


In [40]:
fig_dict['MB5'].show()